# Features Extraction & Clustering. Part II.
This part is focused on clustering inside top-levels clusters that were obtained in the previous part.

## Preparing

First, we have to re-run code of Part I before continuing.

In [1]:
# Import of necessary libs and our classes
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.spatial.distance import squareform, pdist
from sklearn.cluster import DBSCAN
from cj_loader import Storer, Extractor, extract_features
import warnings
warnings.filterwarnings("ignore")

# Init storer object with given data and calculate precense of data
storer = Storer()
precense = storer.applicability()

# Calculation of data completeness matrix and weights
data_compl = storer.data_completeness()
weights = data_compl.mean().sort_values()

# transform boolean matrix to numeric
weighted = precense.copy()
for index, row in precense.iterrows():
    weighted.loc[index][:] = row * weights

# calculate distances between pairs of coins
distances = pd.DataFrame(squareform(pdist(weighted)), index=weighted.index, columns=weighted.index)

# Top-level clustering with DBSCAN algorithm
clustering = DBSCAN(eps=0.3, min_samples=3).fit(distances)
labels = clustering.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print('Estimated number of clusters: %d' % n_clusters_)

# Clusters aggregation
weighted['label'] = labels
clusters = {}
for label in np.unique(labels):
    cl = weighted[weighted['label'] == label]
    clusters[label] = cl

# Extracting features
cl_coin_features = {}
unclustered_coins = set()
for label, cluster in clusters.items():
    if label != -1:
        cl_coin_features[label] = extract_features(storer, '2014-04-01', coins_set=cluster.index)
    else:
        unclustered_coins.update(cluster.index.values.tolist())

Estimated number of clusters: 4


## Adjusting clustering parameters

DBSCAN is one of the most commonly used clustering algorithms and also the most cited in scientific literature. It has a lot of advantages and is widely recommended for various tasks. One of the disatvantages of DBSCAN is routine of parameters selection, especially epsilon parameter. In the previous part, DBSCAN was applied to data having all elements in the \[0;1\] range. That simplified the selection process for epsilon. Our current clustering task is significantly more complex. Parameters are non-normalized and have their own scales. Number of parameters is large, therefore, the task can be considered as high-dimensional clustering. High number of dimensions will also make direct clusters visualization impossible. 

Best possible approach to choosing the correct value for epsilon would be to research results obtained with different values. Fortunately, such method was developed and described in several publications.

[HDBSCAN](https://hdbscan.readthedocs.io/en/latest/index.html) - Hierarchical Density-Based Spatial Clustering of Applications with Noise performs DBSCAN over varying epsilon values and integrates the result to find a clustering that gives the best stability over epsilon. This allows HDBSCAN to find clusters of varying densities (unlike DBSCAN), and be more robust to parameter selection.

In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
import hdbscan

for top_level_cl in range(n_clusters_):
    print("Top-level cluster: {}".format(top_level_cl))
    tc = cl_coin_features[top_level_cl]
    tc = tc.replace([np.inf, -np.inf], np.nan)
    tc = tc.dropna(axis='columns', how='any')
    print("shape: {}".format(tc.shape))
    clusterer = hdbscan.HDBSCAN(min_cluster_size=2)
    ll_labels = clusterer.fit_predict(tc)
    tc['label'] = ll_labels
    for label in np.unique(ll_labels):
        if label != -1:
            print('label %d' %label)
            cl = tc[tc['label'] == label]
            display(cl)
            print('-'*40)
        else:
            noise_coins = tc[tc['label'] == label].index.values.tolist()
            unclustered_coins.update(noise_coins)
            
    print('='*40) 
    
print("Unclustered coins:")
for co in unclustered_coins:
    print(co)

Top-level cluster: 0
shape: (19, 106)
label 0


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,mdiff_to_volatility_mean,mdiff_to_volatility_median,mdiff_to_volatility_stddev,mdiff_to_volatility_skewns,mdiff_to_volatility_kurtos,label
neo,6186.555556,5824.0,6611.747396,1.372165,3.604629,31.586522,20.12,39.027937,1.372236,1.261594,...,2.196671,0.461210,1.907444,6.457338,0.075625,0.000000,0.924062,19.345173,409.596195,0
bitcoin-gold,24770.355731,13296.0,32325.900838,2.885266,9.292075,129.117545,84.71,103.964299,1.048445,0.166518,...,0.225699,0.080849,-0.072862,3.205795,0.565094,0.502647,0.330306,1.074817,1.513131,0
ethereum-classic,10809.951152,10763.0,8286.560932,0.689049,0.725340,12.379725,13.69,10.556761,0.660376,-0.279080,...,2.753299,1.736931,1.205032,0.656309,17.226962,12.174938,79.397441,25.285356,655.311219,0


----------------------------------------
label 1


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,mdiff_to_volatility_mean,mdiff_to_volatility_median,mdiff_to_volatility_stddev,mdiff_to_volatility_skewns,mdiff_to_volatility_kurtos,label
lisk,1256.345088,845.5,1306.419742,2.170264,8.431071,5.084455,1.765000,7.069427,1.846910,2.940810,...,1.780763,0.396637,1.179027,2.528794,234.195165,194.815368,189.347875,1.589927,4.444137,1
pivx,2817.088009,2696.0,1551.385042,1.526174,8.624566,2.080260,1.360000,2.711160,1.755510,3.364241,...,0.187977,0.169446,1.370168,3.121039,41.565378,1.146900,1076.473628,29.339617,858.875856,1
verge,4775.912040,707.0,9105.175995,2.395383,4.998431,0.011389,0.000025,0.029746,3.755064,16.750922,...,0.701068,0.183204,-0.552401,-0.252065,950.513747,33.238570,7896.868751,31.924672,1099.745328,1
gas,4605.768519,4411.5,5576.554873,2.587282,15.329782,23.146736,20.590000,13.790248,1.121104,1.914183,...,1.800429,0.809647,1.883398,3.804189,73.458813,0.013154,197.234739,3.986829,21.984833,1
digibyte,7653.289331,4285.0,7987.288277,2.910194,12.477327,0.007896,0.000279,0.016372,2.991550,11.571083,...,0.640099,0.136387,0.030436,2.460600,1334.301447,122.844788,2645.075160,4.811180,40.258149,1
zcash,53671.197802,56198.0,28965.458710,0.018248,-1.350792,233.724953,220.800000,184.993409,3.025848,20.671637,...,0.096124,0.051123,5.473586,41.712232,2.476253,1.655919,3.959993,4.433368,23.606822,1
dogecoin,44725.500591,35972.0,30990.339987,2.850689,11.276300,0.001132,0.000243,0.001916,3.192562,13.440000,...,0.445320,0.129465,0.957839,0.907888,37756.476612,24201.275609,332187.587638,40.333971,1632.795528,1
decred,10833.712222,11097.5,3816.079987,0.222284,0.659588,29.186445,15.510000,33.340191,0.980179,-0.305222,...,0.302313,0.072040,1.659293,9.949313,59.520629,8.812540,148.366603,5.126046,33.825512,1
waves,4530.174551,3220.0,4783.672582,2.243263,7.484182,3.155054,2.860000,3.291030,1.338110,1.970991,...,2.522097,3.895967,2.982409,12.366264,42.177654,4.054797,159.940076,5.716705,32.118045,1


----------------------------------------
label 2


...

----------------------------------------
label 1


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,trans_per_address_median,trans_per_address_stddev,trans_per_address_skewns,trans_per_address_kurtos,tx_per_address_mean,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,label
populous,449.096606,368.0,289.220522,2.592868,11.805822,16.097421,11.455000,15.257558,1.439948,1.470089,...,5537.818441,6190.729083,1.694793,3.006970,1.216695,1.184783,0.237509,6.244834,68.613779,1
zilliqa,730.729592,596.5,817.031316,6.584975,52.120121,0.081106,0.071365,0.035365,0.969616,0.274079,...,16241.574272,26118.927198,7.647189,75.329836,1.400679,1.372200,0.324885,0.265146,-0.756347,1
rchain,65.275304,47.5,74.429443,2.578367,13.607838,1.094281,1.050000,0.678605,0.335606,-0.615536,...,5708.825768,249371.808790,11.218519,139.014991,0.897106,0.902950,0.215825,0.848830,3.865996,1
aelf,410.696833,257.0,1106.880388,13.475649,190.262334,1.091667,1.050000,0.451269,0.740229,-0.127404,...,15626.563230,42210.151964,6.545389,53.551399,1.499277,1.363958,0.440857,1.578175,3.187754,1
qash,259.127586,184.0,227.125618,2.312331,7.983496,0.764087,0.689685,0.423743,1.446614,2.546533,...,5886.808622,24812.639626,6.688334,49.751077,1.018055,1.016605,0.186535,0.067582,0.897683,1
ethos,323.564557,278.0,208.521250,2.368344,9.206586,2.256412,1.700000,1.835292,1.707954,3.736363,...,4224.490677,5869.650188,5.926127,60.513508,1.043667,1.016506,0.153906,0.727335,1.039820,1
augur,456.766484,377.0,323.064070,3.191605,18.721484,20.261361,15.850000,19.904376,1.715515,3.295499,...,5483.098352,15132.520550,6.322831,47.898602,1.385219,1.304811,0.451716,6.304069,48.630149,1
funfair,568.948052,297.0,630.972189,2.180213,4.649262,0.039460,0.029683,0.029096,2.533569,7.665132,...,3075.443915,86043.307517,14.154505,221.879295,1.449644,1.233333,0.659184,2.553176,7.034447,1
kucoin-shares,216.581121,15.0,1513.373280,11.114239,144.287287,3.680672,2.990000,3.411707,2.522776,7.750430,...,1353.139363,964460.491300,12.316858,155.735274,0.926963,0.869565,0.341569,4.804123,34.968228,1
aion,350.647059,242.0,410.546863,3.701675,15.264639,2.691735,2.355000,1.888573,1.611819,3.118996,...,8500.707163,155013.764539,13.618181,202.178100,1.163089,1.126638,0.201389,1.502889,4.640763,1


----------------------------------------
label 2


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,trans_per_address_median,trans_per_address_stddev,trans_per_address_skewns,trans_per_address_kurtos,tx_per_address_mean,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,label
binance-coin,999.766234,217.0,7791.168444,14.382106,226.810603,8.084689,9.26,5.893013,0.043746,-1.360099,...,5051.448981,23890.993978,6.502648,49.615987,1.082846,1.062092,0.193961,0.827556,2.178119,2
omisego,3904.139896,2467.5,8247.695864,6.691323,49.809485,10.894890,9.82,4.869719,0.360876,0.131471,...,7600.454755,18506.548801,13.566305,214.684558,1.375349,1.256275,0.413903,4.556722,30.751261,2


----------------------------------------
Unclustered coins:
factom
tron
mixin
bitcoin
elastos
veritaseum
ontology
gifto
iostoken
litecoin
bitcoin-cash
tether
huobi-token
stratis
cardano
kin
smartcash
monero
qtum
pundi-x
iota
nebulas-token
tezos
ripple
ethereum
cryptonex
mithril
nano
nxt
eos
stellar
tenx


## Important notes
### Approach limitations
Implemented approach is very general. It doesn't detect any mutual dependencies, following trends and other complex elements. Therefore, it should be considered as clustering by basic time series characteristics. Particularly, top-level clustering is a rather necessary measure than the usual taken step. List of features describing each time serie is quite short, but it can easily be extended with additional features.
### Label "-1"
HDBSCAN (as extension of DBSCAN method) groups together points that are closely packed together (points with many nearby neighbors), marking as outliers points that lie alone in low-density regions (whose nearest neighbors are too far away). This leads to appearing a section named "Unclustered coins" which includes these outliers belonging to -1 label. This list consists of two types of coins: coins with big amount of data missing and **real** outliers. Each coin presented in this list should be considered as **special**. it's not surprising, therefore, that couple of crypto-headliners appeared there: they are really outliers by many of the parameters.
### Metrics
HDBSCAN [can use different metrics][0] to calculate distance between coins by their values of features. The case examined uses default Euclidean metrics. Main disadvantage of such metrics is assumption that different parameters equally influence the distance. That actually is not true. The question of choosing metrics for high-dimensional clustering is [very][1] [controversial][2]. Generally one of the best ways is to define weight for each feature and use weighted Euclidean distance. Possibly this can be the theme of separate (and pretty complex) research.
### Methods
Of course there are a lot of clustering algorithms nowadays that can be applied for this task instead of (H)DBSCAN. But most probably, applying of described approach to data splitted by timeframes of different market states as well as widening feature-sets extraction method can deliver more significant results. Next proposed step is switching to tsfresh library for automated features extraction. 
[0]:https://hdbscan.readthedocs.io/en/latest/basic_hdbscan.html#what-about-different-metrics
[1]:https://www.researchgate.net/post/What_is_the_best_distance_measure_for_high_dimensional_data
[2]:https://stats.stackexchange.com/questions/99171/why-is-euclidean-distance-not-a-good-metric-in-high-dimensions